#  LangChain의 개념과 주요 컴포넌트 이해

---

## 1. LangChain 소개

### 1.1 LangChain이란?
**LangChain**은 대화형 AI 애플리케이션을 쉽게 개발할 수 있도록 도와주는 프레임워크입니다.

#### 📦 핵심 가치
- **모듈화**: 독립적인 컴포넌트를 조합하여 복잡한 AI 시스템 구축
- **상호운용성**: 다양한 AI 모델과 데이터 소스를 하나의 인터페이스로 통합
- **확장성**: 간단한 챗봇부터 복잡한 AI 에이전트까지 확장 가능
- **관찰성**: LangSmith를 통한 실시간 모니터링 및 디버깅

#### 📦 핵심 아키텍처
```markdown
**LangChain 생태계**
├── langchain-core     # 기본 추상화 및 인터페이스
├── langchain         # 체인, 에이전트, 검색 전략
├── langchain-openai  # OpenAI 통합
├── langchain-anthropic # Anthropic 통합
├── LangGraph        # 복잡한 에이전트 워크플로우
└── LangSmith        # 모니터링 및 디버깅
```

<div style="text-align: center;">
    <img src="https://python.langchain.com/svg/langchain_stack_112024_dark.svg" 
        alt="langchain_stack" 
        width="600" 
        style="border: 0;">
</div>

---

## 2. 환경 설정

### 2.1 설치

```bash
# pip 설치
pip install langchain langchain-openai langchain-google-genai

# uv 설치 
uv add langchain langchain-openai langchain-google-genai

# 추가 도구 pip 설치 (선택사항)
pip install langchain-ollama langsmith

# 추가 도구 uv 설치 (선택사항)
uv add langchain-ollama langsmith
``` 

### 2.2 API 키 설정
```python
# .env 파일 생성
OPENAI_API_KEY=your_openai_api_key_here
GOOGLE_API_KEY=your_google_api_key_here

# LangSmith 설정 (선택사항)
LANGSMITH_API_KEY=your_langsmith_api_key
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=your_project_name
```

In [1]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# LangSmith 추적 확인
import os
print(f"LangSmith 추적: {os.getenv('LANGSMITH_TRACING')}")

LangSmith 추적: true


---

## 3. 핵심 컴포넌트

### 3.1 Chat Models (채팅 모델)

- OpenAI, Anthropic, Google 등 다양한 모델을 지원
- 텍스트 생성, 대화, 요약 등의 작업을 수행

In [3]:
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.3,
    top_p=0.95
)

# 간단한 대화
response = model.invoke("탄소의 원자 번호는 무엇인가요?")
print(f"답변: {response.content}")
print(f"메타데이터: {response.response_metadata}")

답변: 탄소의 원자 번호는 6입니다.
메타데이터: {'token_usage': {'completion_tokens': 11, 'prompt_tokens': 18, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C1O5PzW0oL1NpoyRphKWyhrwg7DYm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}


In [4]:
response

AIMessage(content='탄소의 원자 번호는 6입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 18, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C1O5PzW0oL1NpoyRphKWyhrwg7DYm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3c3b8a58-8699-40f9-af4a-283101f0f09f-0', usage_metadata={'input_tokens': 18, 'output_tokens': 11, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 3.2 Messages (메시지)

- 메시지는 AI와의 대화에서 역할을 구분하는 기본 단위입니다.
- 메시지는 사용자, AI, 시스템 등 다양한 역할을 가질 수 있습니다.

In [5]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# 시스템 메시지: AI의 역할 정의
system_msg = SystemMessage(content="당신은 친절한 화학 선생님입니다.")

# 사용자 메시지
human_msg = HumanMessage(content="탄소의 원자 번호는 몇 번인가요?")

# 대화 실행
messages = [system_msg, human_msg]
response = model.invoke(messages)
print(response.content)

탄소의 원자 번호는 6번입니다. 도움이 필요하면 언제든지 물어보세요!


### 3.3 Prompt Templates (프롬프트 템플릿)

- 템플릿을 사용하여 일관된 프롬프트를 생성할 수 있습니다.
- 변수 치환을 통해 동적인 프롬프트를 적용하는 데 유용합니다.


#### 📦 기본 템플릿

In [6]:
from langchain_core.prompts import PromptTemplate

# 전문가 템플릿
template = """
당신은 {topic} 분야의 전문가입니다. {topic}에 관한 다음 질문에 답변해주세요.
질문: {question}
답변: """

prompt = PromptTemplate.from_template(template)

# 템플릿 입력 변수 확인
print(f"필수 변수: {prompt.input_variables}")

필수 변수: ['question', 'topic']


In [7]:
# 템플릿 확인
print(f"템플릿: {prompt.template}")

템플릿: 
당신은 {topic} 분야의 전문가입니다. {topic}에 관한 다음 질문에 답변해주세요.
질문: {question}
답변: 


In [8]:
# 템플릿 사용
formatted_prompt = prompt.format(
    topic="화학",
    question="탄소의 원자 번호는 무엇인가요?"
)
print(formatted_prompt)


당신은 화학 분야의 전문가입니다. 화학에 관한 다음 질문에 답변해주세요.
질문: 탄소의 원자 번호는 무엇인가요?
답변: 


#### 📦 채팅 템플릿

In [9]:
from langchain_core.prompts import ChatPromptTemplate

# 채팅용 템플릿
chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 전문 {subject} 상담사입니다."),
    ("human", "{question}")
])

# 템플릿 사용
prompt = chat_template.invoke({
    "subject": "진로",
    "question": "데이터 사이언티스트가 되려면 어떤 공부를 해야 하나요?"
})

In [10]:
# 템플릿 확인
print(prompt)

messages=[SystemMessage(content='당신은 전문 진로 상담사입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='데이터 사이언티스트가 되려면 어떤 공부를 해야 하나요?', additional_kwargs={}, response_metadata={})]


In [11]:
# 메시지 속성 확인
for message in prompt.messages:
    print(f"역할: {message.type}, 내용: {message.content}")

역할: system, 내용: 당신은 전문 진로 상담사입니다.
역할: human, 내용: 데이터 사이언티스트가 되려면 어떤 공부를 해야 하나요?


---

## 4. LCEL (LangChain Expression Language)

### 4.1 LCEL이란?
**LCEL**은 `|` 연산자를 사용하여 컴포넌트들을 순차적으로 연결하는 선언적 체이닝을 지원합니다.

#### 📦 핵심 특징
- **재사용성**: 정의된 체인을 다른 체인의 컴포넌트로 활용
- **다양한 실행 방식**: `.invoke()`, `.batch()`, `.stream()`, `.astream()`
- **자동 최적화**: 배치 처리 시 효율적인 작업 수행
- **스키마 지원**: 입력/출력 스키마 자동 생성

### 4.2 기본 체인 구성

#### 📦 Prompt + LLM

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 컴포넌트 정의
prompt = PromptTemplate.from_template(
    "당신은 {topic} 분야의 전문가입니다. {topic}에 관한 다음 질문에 답변해주세요.\n"
    "질문: {question}"
)
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)

# 체인 구성
chain = prompt | llm

# 체인 실행
response = chain.invoke({
    "topic": "화학",
    "question": "탄소의 원자 번호는 무엇인가요?"
})

print(f"답변: {response.content}")

답변: 탄소의 원자 번호는 6입니다.


#### 📦 Prompt + LLM + Output Parser

In [17]:
from langchain_core.output_parsers import StrOutputParser

# 출력 파서 추가
output_parser = StrOutputParser()

# 완전한 체인 구성
chain = prompt | llm | output_parser

# 체인 실행 (문자열 반환)
response = chain.invoke({
    "topic": "화학",
    "question": "탄소의 원자 번호는 무엇인가요?"
})

print(f"답변: {response}")  # 이제 문자열로 반환

답변: 탄소의 원자 번호는 6입니다.


---

## 5. LangSmith 모니터링

### 5.1 LangSmith란?
**LangSmith**는 LLM 애플리케이션의 관찰성(Observability)을 제공하는 도구입니다.

#### 📦 주요 기능
- **체인 실행 로깅 및 추적**
- **프롬프트 디버깅**
- **성능 측정 및 분석**
- **실시간 모니터링**

### 5.2 LangSmith 설정

#### 📦 계정 가입 및 설정
```python
# 1. LangSmith 계정 가입: https://www.langchain.com/langsmith
# 2. .env 파일 설정
LANGSMITH_API_KEY=your_langsmith_api_key
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=your_project_name

# 3. 환경 확인
from dotenv import load_dotenv
import os

load_dotenv()
print(f"LangSmith 추적 상태: {os.getenv('LANGSMITH_TRACING')}")
print(f"프로젝트명: {os.getenv('LANGCHAIN_PROJECT')}")
```

### 5.3 자동 추적

LangSmith가 설정되면 모든 체인 실행이 자동으로 추적됩니다:

```python
# 체인 실행 시 자동으로 LangSmith에 로그 전송
chain = prompt | llm | StrOutputParser()

response = chain.invoke({
    "topic": "인공지능",
    "question": "머신러닝과 딥러닝의 차이점은?"
})

# LangSmith 대시보드에서 실행 과정 확인 가능:
# - 각 단계별 실행 시간
# - 입력/출력 데이터
# - 토큰 사용량
# - 에러 추적
```


---

## 6. Runnable 인터페이스

### 6.1 Runnable 개념
모든 LangChain 컴포넌트는 **Runnable 인터페이스**를 구현하여 일관된 방식으로 실행됩니다.

#### 📦 주요 Runnable 유형
- `RunnableSequence`: 순차적 실행
- `RunnableParallel`: 병렬 실행
- `RunnablePassthrough`: 입력 전달
- `RunnableLambda`: 함수 래핑

### 6.2 RunnableSequence (순차 실행)

In [18]:
# 번역 전용 체인 (파이프 연산자 사용)
translation_prompt = PromptTemplate.from_template(
    "'{text}'를 영어로 번역해주세요. 번역된 문장만을 출력해주세요."
)

translation_chain = translation_prompt | llm | StrOutputParser()

# 번역 실행
result = translation_chain.invoke({"text": "좋은 하루 되세요!"})
print(result)

Have a nice day!


In [19]:
from langchain_core.runnables import RunnableSequence

# 명시적 RunnableSequence 생성
translation_chain = RunnableSequence(
    first=translation_prompt, 
    middle=[llm],
    last=StrOutputParser()
)

# 파이프 연산자와 동일한 결과
# translation_chain = translation_prompt | llm | StrOutputParser()

result = translation_chain.invoke({"text": "좋은 하루 되세요!"})
print(result)

Have a nice day!


### 6.3 RunnableParallel (병렬 실행)

#### 📦 질문 분석 시스템

In [20]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

# 1. 주제 분류 체인
topic_template = """
다음 카테고리 중 하나로 분류하세요:
- 화학(Chemistry)
- 물리(Physics)  
- 생물(Biology)

질문: {question}
분류: """

topic_prompt = PromptTemplate.from_template(topic_template)
topic_chain = topic_prompt | llm | StrOutputParser()

# 2. 언어 감지 체인
language_template = """
입력된 텍스트의 언어를 분류하세요:
- 한국어(Korean)
- 영어(English)
- 기타(Others)

입력: {question}
언어: """

language_prompt = PromptTemplate.from_template(language_template)
language_chain = language_prompt | llm | StrOutputParser()

# 3. 답변 생성 체인
answer_template = """
당신은 {topic} 분야의 전문가입니다. 
{language}로 다음 질문에 답변해주세요.

질문: {question}
답변: """

answer_prompt = PromptTemplate.from_template(answer_template)

# 4. 병렬 처리 체인 구성
analysis_chain = RunnableParallel({
    "topic": topic_chain,
    "language": language_chain,
    "question": itemgetter("question")
})

# 5. 전체 체인 연결
complete_chain = analysis_chain | answer_prompt | llm | StrOutputParser()

# 실행
result = complete_chain.invoke({
    "question": "탄소의 원자 번호는 무엇인가요?"
})
print(f"최종 답변: {result}")

최종 답변: 답변: 탄소의 원자 번호는 6입니다.


---

## 7. **실전 프로젝트**: 감정 분석 파이프라인 ~ 13:40분까지


In [21]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

# 요약 프롬프트
summarize_prompt = PromptTemplate.from_template(
    "다음 텍스트를 한 문장으로 요약해주세요: {text}"
)

# 감정 분석 프롬프트
sentiment_prompt = PromptTemplate.from_template("""
다음 텍스트의 감정을 분석해주세요.

텍스트: {summary}

규칙:
1. 반드시 '긍정', '부정', '중립' 중 하나의 단어로만 답변하세요
2. 다른 설명이나 부가 정보는 포함하지 마세요

답변:
""")

# 체인 구성
summarize_chain = summarize_prompt | llm
sentiment_chain = sentiment_prompt | llm | StrOutputParser()

# 전체 파이프라인
analysis_pipeline = (
    summarize_chain 
    | RunnableParallel(
        summary=lambda x: x.content,
        sentiment=lambda x: sentiment_chain.invoke({"summary": x.content}),
    )
)

# 테스트 텍스트
text = """오늘 시험을 봤는데 정말 잘 본 것 같아요. 
몇 주 동안 열심히 공부한 보람이 있네요. 
결과가 나오면 좋은 점수를 받을 수 있을 것 같아서 기대됩니다."""

result = analysis_pipeline.invoke({"text": text})
print(f"요약: {result['summary']}")
print(f"감정 분석: {result['sentiment']}")

요약: 오늘 시험을 열심히 공부한 덕분에 잘 봤고 좋은 결과를 기대하고 있습니다.
감정 분석: 긍정


---

## 8. 실습 문제  ~13:50

#### 🌟 문제 1: LCEL 기본 체인 만들기
- 사용자가 입력한 주제에 대해 3줄 요약 제공
- 온도는 0.5로 설정
- 문자열 출력 파서 사용

In [ ]:
# 여기에 코드를 작성하세요
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


#### 🌟 문제 2: RunnableParallel 활용
- 하나의 입력에 대해 번역과 요약을 동시에 수행하는 체인을 만드세요.


In [ ]:
# 여기에 코드를 작성하세요
from langchain_core.runnables import RunnableParallel



---

## 🔗 유용한 링크

- [LangChain 공식 문서](https://python.langchain.com/docs/introduction/)
- [LangSmith 가이드](https://docs.smith.langchain.com/)
- [LangChain GitHub](https://github.com/langchain-ai/langchain)
- [LangChain Academy](https://academy.langchain.com/)

---

In [ ]:
### 실습 8-1 답안

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

summary_prompt = PromptTemplate.from_template(
    "{topic}에 대해 3줄로 요약해주세요."
)

model = ChatOpenAI(model="gpt-4.1-mini", temperature=0.5)
output_parser = StrOutputParser()

summary_chain = summary_prompt | model | output_parser

# 테스트
result = summary_chain.invoke({"topic": "인공지능"})
print(result)

In [ ]:
### 실습 8-2 답안

from langchain_core.runnables import RunnableParallel

# 번역 체인
translate_prompt = PromptTemplate.from_template(
    "다음 텍스트를 영어로 번역하세요: {text}"
)
translate_chain = translate_prompt | model | StrOutputParser()

# 요약 체인  
summarize_prompt = PromptTemplate.from_template(
    "다음 텍스트를 한 문장으로 요약하세요: {text}"
)
summarize_chain = summarize_prompt | model | StrOutputParser()

# 병렬 처리 체인
parallel_chain = RunnableParallel({
    "translation": translate_chain,
    "summary": summarize_chain
})

# 테스트
result = parallel_chain.invoke({
    "text": "안녕하세요. 오늘은 정말 좋은 날씨입니다. 산책하기 딱 좋네요."
})
print(f"번역: {result['translation']}")
print(f"요약: {result['summary']}")